# Notes
1. I did not have the time to make the preprocessing steps in one class so I used 9 functions.
2. The preprocessing steps have 3 cells 
3. There is a function on the __2nd preprocessing cell__ that translates texts (it detected 60 texts but only 20 were non english).
4. If it is considered prohibited you can exclude (it does not change the cross validation accuracy at all). I added a Note to differentiate easier.

__Important:__ It is mentioned on the moodle to create a function named train_test(train.csv, test.csv). I was confused , and I just read train.csv and test.csv in a pandas dataframe, considering that you will add test.csv on the folder that i will submit which contains my .pynb files and train.csv. If you don't add  test.csv on the folder that i will give you, please change the path on train and test dataframes.

# Install the libraries

In [3]:
pip install beautifulsoup4

In [272]:
pip install wordninja  #this splits words and is used only on hashtags e.g #BestHolidays -> Best Holidays

##### Don't install these if translating is considered prohibited.

In [ ]:
#pip install langdetect
#pip install googletrans==4.0.0-rc1

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import wordninja
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Read train.csv and test.csv

In [3]:
# If train.csv and test.csv are added on the folder with this notebook
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'test.csv'

In [ ]:
train.shape

In [ ]:
train.head(1)

In [ ]:
test.head(1)

In [ ]:
test.shape

#### If the test does not have the same column names with train please rename it to text

# Preprocessing

In [ ]:
def remove_URL_HTML(text):
    # Remove URLs
    text = re.sub(r"http.\S+", '', text)
    text = re.sub(r"www.\S+", '', text)
    text = re.sub(r"pic.twitter.com\S+", '', text)
    # Remove HTML/XML tags
    if re.search(r'<.*?>', text):  
        text = BeautifulSoup(text, "html.parser").get_text()  
        
    return text

train['cleaned_text'] = train['text'].apply(remove_URL_HTML)
test['cleaned_text'] = test['text'].apply(remove_URL_HTML)

def remove_tags_and_split_hashtags(text, keep_hashtags=True):
    # Remove @USER (e.g., @user1, @someone)
    text = re.sub(r'@[\w]+', '', text)

    if keep_hashtags:
        # Find all hashtags (e.g., #something)
        hashtags = re.findall(r'#\w+', text)
        
        # Replace hashtags with their split version
        for hashtag in hashtags:
            # Remove the '#' and split the word into components
            split_words = ' '.join(wordninja.split(hashtag[1:]))
            text = text.replace(hashtag, split_words)
    else:
        # Remove hashtags completely
        text = re.sub(r'#\w+', '', text)
    
    return text

train['cleaned_text'] = train['cleaned_text'].apply(lambda x: remove_tags_and_split_hashtags(x, keep_hashtags=True))
test['cleaned_text'] = test['cleaned_text'].apply(lambda x: remove_tags_and_split_hashtags(x, keep_hashtags=True))

# This is a custom dict
emoji_dict = {
    r':-\)': ' happy face ',
    r':\)': ' happy face ',
    r':D': ' laugh face ',
    r':-\(': ' sad face ',
    r':\(': ' sad face ',
    r':O': ' surprised face ',
    r';-\)': ' wink face ',
    r';\)': ' wink face ',
    r':P': ' playful face ',
    r':-\?': ' undecided face ',
    r':/': ' undecided face ',
    r"\b't\b": " not" # I added this to keep all the verbs with not e.g can't -> cannot
}  

def replace_patterns_with_meaning(text, emoji_dict, replace=True):
    if replace:
        for emoji, meaning in emoji_dict.items():
            text = re.sub(emoji, meaning, text)
    return text

train['cleaned_text'] = train['cleaned_text'].apply(lambda x: replace_patterns_with_meaning(x, emoji_dict))
test['cleaned_text'] = test['cleaned_text'].apply(lambda x: replace_patterns_with_meaning(x, emoji_dict))

# This is from Kaggle I added this on Report's references.
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "af" : "as fuck",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",   
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "c'mon" : "come on",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

def replace_abbreviations(text, abbreviations, replace=True):
    if replace:
        words = text.split()
        replaced_words = [abbreviations.get(word.lower(), word) for word in words] 
    return " ".join(replaced_words)

train['cleaned_text'] = train['cleaned_text'].apply(lambda x: replace_abbreviations(x, abbreviations))
test['cleaned_text'] = test['cleaned_text'].apply(lambda x: replace_abbreviations(x, abbreviations))

def preprocess_text(text):
    # Convert text to lowercase, remove punctuation and extra spaces
    text = text.lower()  
    text = re.sub(r'\d+', ' ', text) 
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text) 
#    text = re.sub(r'[^!\?\'\w\s]', '', text)  # Keep '!', '?'??????????????? to capture more expressions??
    return text

train['cleaned_text'] = train['cleaned_text'].apply(preprocess_text)
test['cleaned_text'] = test['cleaned_text'].apply(preprocess_text)



## Note (translate to english)

In [ ]:

# Skip this part if this is considered pre-trained

In [ ]:
from googletrans import Translator
from langdetect import detect

translator = Translator()

# Function to detect language and translate to English
def translate_to_english(text):
    try:
        language = detect(text)
        if language != 'en':
            translated_text = translator.translate(text, src=language, dest='en').text
            return translated_text
        else:
            return text 
    except Exception as e:
        print(f"Error: {e}")
        #return original text in case of any error
        return text


train['cleaned_text'] = train['cleaned_text'].apply(translate_to_english)
test['cleaned_text'] = test['cleaned_text'].apply(translate_to_english)

In [ ]:
def stopwords_and_stem(text, stopwords=False):
    stemmer = PorterStemmer()
    tokens = word_tokenize(text)  
    if stopwords:
        stemmed_tokens = [stemmer.stem(word.lower()) for word in tokens if word.lower() not in stop_words]
    else:
        stemmed_tokens = [stemmer.stem(word.lower()) for word in tokens]
    return " ".join(stemmed_tokens)


train['cleaned_text'] = train['cleaned_text'].apply(stopwords_and_stem)
test['cleaned_text'] = test['cleaned_text'].apply(stopwords_and_stem)

def remove_short_words(text):
    
    words = text.split()  
    filtered_words = [word for word in words if len(word) >= 2]  
    return ' '.join(filtered_words)  

train['cleaned_text'] = train['cleaned_text'].apply(remove_short_words)
test['cleaned_text'] = test['cleaned_text'].apply(remove_short_words)

def preprocess_text_with_token_limit(text, token_limit=200, token_cut=100):

    tokens = word_tokenize(text)
    if len(tokens) > token_limit:
        tokens = tokens[:token_cut] + tokens[-token_cut:]
    
    return ' '.join(tokens)

train['cleaned_text'] = train['cleaned_text'].apply(preprocess_text_with_token_limit)
test['cleaned_text'] = test['cleaned_text'].apply(preprocess_text_with_token_limit)

# Train

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import normalize
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [ ]:
class ConditionalPowerNormalization(BaseEstimator, TransformerMixin):
    def __init__(self, transform_power_norm=True):
        self.transform_power_norm = transform_power_norm

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if self.transform_power_norm:
            X.data **= 0.5  # Apply square root transformation 
            # Normalize the sparse matrix
            normalize(X, copy=False)
        return X

In [ ]:
# Create pipelines with best parameters of tfi-df/countervec for each model
linear_svc_pipeline = make_pipeline(
    TfidfVectorizer(max_df=0.3, max_features=None, min_df=1, ngram_range=(1, 2), norm='l1', sublinear_tf=False), 
    ConditionalPowerNormalization(transform_power_norm=True),
    LinearSVC(C=0.5, class_weight='balanced', loss='squared_hinge')
)

multinomial_nb_pipeline = make_pipeline(
    CountVectorizer(max_df=0.25, min_df=2, ngram_range=(1, 2)),  
    MultinomialNB(alpha=1, fit_prior=True)
)

logistic_reg_pipeline = make_pipeline(
    TfidfVectorizer(max_df=0.45, min_df=2, ngram_range=(1, 2), sublinear_tf=True),  
    LogisticRegression(C=1, multi_class='multinomial', penalty='l2', solver='lbfgs')
)

# Voting classifier 
voting_clf = VotingClassifier(estimators=[
    ('linear_svc', linear_svc_pipeline),
    ('multinomial_nb', multinomial_nb_pipeline),
    ('logistic_reg', logistic_reg_pipeline)
], voting='hard')

In [ ]:
X_train = train['cleaned_text']
y_train = train['sentiment']

voting_clf.fit(X_train, y_train)

# Predict

In [ ]:
X_test = test['cleaned_text']
#y_test = test['sentiment']   
X_test.shape

In [ ]:
predictions = voting_clf.predict(X_test)

In [ ]:
predictions[:10]

# predictions.txt

In [102]:
with open('predictions.txt', 'w') as f:
    for pred in predictions:
        f.write(str(pred) + '\n')  

### Ignore this i was validating the entire procedure on 20 custom texts.

In [119]:
#Test on 20 custom texts
y_pred = voting_clf.predict(test['cleaned_text'])
test_accuracy = accuracy_score(test['sentiment'], y_pred)
test_accuracy

0.85

In [121]:
test['cleaned_text'].shape

(20,)

In [123]:
y_pred.shape

(20,)

In [130]:
y_pred == predictions

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [ ]:
cv_scores = cross_val_score(voting_clf, X_train, y_train, cv=StratifiedKFold(n_splits=8, shuffle=True))

In [126]:
np.mean(cv_scores)

0.603875

In [128]:
cv_scores

array([0.59 , 0.611, 0.613, 0.596, 0.63 , 0.583, 0.595, 0.613])